In [2]:
import pandas as pd
import numpy as np
import datetime
import logger

In [5]:
dates = []
date  = datetime.datetime(2019, 5, 15)

for i in range(17):

    if(i%2 == 0):
        ndate = date + datetime.timedelta(hours=0, minutes = 0, seconds = 20)
    elif(i%3 == 0):
        ndate = date - datetime.timedelta(hours=0, minutes = 0, seconds = 2)
    elif(i%4 == 0):
        ndate = date - datetime.timedelta(hours=0, minutes = 0, seconds = 10)
    elif(i%5 == 0):
        ndate = date - datetime.timedelta(hours=0, minutes = 0, seconds = 3)
    else:
        ndate = date + datetime.timedelta(hours=0 ,minutes=0, seconds = 0)
        
    dates.append(ndate)
    date = ndate

dates

[datetime.datetime(2019, 5, 15, 0, 0, 20),
 datetime.datetime(2019, 5, 15, 0, 0, 20),
 datetime.datetime(2019, 5, 15, 0, 0, 40),
 datetime.datetime(2019, 5, 15, 0, 0, 38),
 datetime.datetime(2019, 5, 15, 0, 0, 58),
 datetime.datetime(2019, 5, 15, 0, 0, 55),
 datetime.datetime(2019, 5, 15, 0, 1, 15),
 datetime.datetime(2019, 5, 15, 0, 1, 15),
 datetime.datetime(2019, 5, 15, 0, 1, 35),
 datetime.datetime(2019, 5, 15, 0, 1, 33),
 datetime.datetime(2019, 5, 15, 0, 1, 53),
 datetime.datetime(2019, 5, 15, 0, 1, 53),
 datetime.datetime(2019, 5, 15, 0, 2, 13),
 datetime.datetime(2019, 5, 15, 0, 2, 13),
 datetime.datetime(2019, 5, 15, 0, 2, 33),
 datetime.datetime(2019, 5, 15, 0, 2, 31),
 datetime.datetime(2019, 5, 15, 0, 2, 51)]

In [8]:
df = pd.DataFrame(np.random.randn(17, 4), index=dates, columns=['A', 'B', 'C', 'D'])
print(df)
print(df.columns)

                            A         B         C         D
2019-05-15 00:00:20 -0.960066  0.413707 -1.377400 -1.002626
2019-05-15 00:00:20 -0.886869  0.422776 -1.106182 -1.486319
2019-05-15 00:00:40 -0.096105 -0.102055  0.276345 -0.782738
2019-05-15 00:00:38 -0.126549 -0.575007 -0.511843 -1.336956
2019-05-15 00:00:58 -0.014829  1.113025  0.914659  0.331295
2019-05-15 00:00:55 -2.124429 -1.138002  0.224648 -0.049879
2019-05-15 00:01:15 -1.201261  1.006790  2.551382 -0.358086
2019-05-15 00:01:15 -0.560281  0.086548 -0.392928  1.421808
2019-05-15 00:01:35 -0.724767 -0.533443 -0.279261  0.854617
2019-05-15 00:01:33  1.110426  0.449788 -0.929471  1.019215
2019-05-15 00:01:53 -1.404215 -1.855989  0.000110 -0.676832
2019-05-15 00:01:53 -0.112373  0.688080 -0.515549  1.124070
2019-05-15 00:02:13 -0.330233 -0.109917  0.032900 -1.070363
2019-05-15 00:02:13  0.548199  0.517943  0.640609  0.278517
2019-05-15 00:02:33 -0.651787 -0.121548  0.579289 -0.330294
2019-05-15 00:02:31  0.585318 -1.702853 

In [10]:
def find_sampling_freq(ts):
    # assert ts.index is datetime object
    try:
        # For non-NA values, check time difference in seconds in each timestamp record for each metric
        sf = pd.Series(ts.index).diff().dropna().dt.seconds
        # logger.info('Info: Found natural sampling frequency')
    except:
        logger.exception('Error: Cannot find natural sampling frequency.')
        
    # Only keep record with timestamp difference greater than 0 seconds
    sf = sf[sf > 0]
    # if the series has different timestamp, take the mininum value from the mode as the sampling frequency
    if len(sf) > 0:
        sampling_frequency = int(sf.mode().values.min())
    # Otherwise, sampling frequency is 0
    else:
        sampling_frequency = 0
    return ts, sampling_frequency

In [13]:
samp_freqs = find_sampling_freq(df['A'])

In [26]:
for i in range(len(samp_freqs)):
    print(samp_freqs[i])

2019-05-15 00:00:20   -0.960066
2019-05-15 00:00:20   -0.886869
2019-05-15 00:00:40   -0.096105
2019-05-15 00:00:38   -0.126549
2019-05-15 00:00:58   -0.014829
2019-05-15 00:00:55   -2.124429
2019-05-15 00:01:15   -1.201261
2019-05-15 00:01:15   -0.560281
2019-05-15 00:01:35   -0.724767
2019-05-15 00:01:33    1.110426
2019-05-15 00:01:53   -1.404215
2019-05-15 00:01:53   -0.112373
2019-05-15 00:02:13   -0.330233
2019-05-15 00:02:13    0.548199
2019-05-15 00:02:33   -0.651787
2019-05-15 00:02:31    0.585318
2019-05-15 00:02:51   -0.668249
Name: A, dtype: float64
20


In [29]:
def aggregate_ts(ts, orig_frequency, new_frequency):
    # Compare the calculated sampling frequency with the defined re-sample frequency
    # resample
    print(orig_frequency)
    print(new_frequency)
    if orig_frequency <= new_frequency:
        # print(ts.name, new_frequency)
        if ts.dtypes != 'object':
            ts_new = ts.resample(str(new_frequency)+'s').first().interpolate()
        else:
            ts_new = ts.resample(str(new_frequency)+'s').first()
    else:
        # print(ts.name, orig_frequency)
        if ts.dtypes != 'object':
            ts_new = ts.resample(str(orig_frequency)+'s').first().interpolate()
        else:
            ts_new = ts.resample(str(orig_frequency)+'s').first()
    # print(ts_new.name, ts_new.dtypes)
    return ts_new

In [30]:
aggregate_ts(samp_freqs[0],samp_freqs[1],30)

20
30


2019-05-15 00:00:00   -0.960066
2019-05-15 00:00:30   -0.126549
2019-05-15 00:01:00   -1.201261
2019-05-15 00:01:30    1.110426
2019-05-15 00:02:00   -0.330233
2019-05-15 00:02:30    0.585318
Freq: 30S, Name: A, dtype: float64